In [2]:
import pandas as pd
from glob import glob
d = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/csv/*.csv"
fnames = glob(d)
print(fnames[0])

/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/csv/FAST_api_data_2010-01-01_2024-11-26.csv


In [4]:
fpath = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/csv/<TIC>_api_data_2010-01-01_2024-11-26.csv"

In [11]:
working_tic = "AALI"
working_path = fpath.replace("<TIC>", working_tic)
working_df = pd.read_csv(sample_path)
working_dates = sample_df["date"].unique().tolist()
len(working_dates)

3627

In [ ]:
date2day_index = {date: dy for dy, date in enumerate(working_dates, start=1)}
date2day_index

In [6]:
flist = "tuntun_scripts/tics_to_repeat_fetch.txt"
list_to_redo_fetch = open(flist).read().splitlines()
print(list_to_redo_fetch)
print(len(list_to_redo_fetch))

['IBFN', 'ANDI', 'BELL', 'ENVY', 'CBPE', 'BIRD', 'BIKE', 'CHIP', 'BBYB', 'GRPH', 'HRME', 'GOLD', 'BIPI', 'HALO', 'CARS', 'GPSO', 'FOLK', 'CSIS', 'APEX', 'ARII', 'BEER', 'GIAA', 'BIKA', 'DRMA', 'DIGI', 'BINA', 'AMAN', 'APLN', 'BLTZ', 'CAMP', 'DAYA', 'BOLT', 'BANK', 'CSMI', 'GOTO', 'AGII', 'FILM', 'DMAS', 'CSRA', 'CMNT', 'FAPA', 'GRPM', 'HOMI', 'BTPS', 'BYAN', 'BULL', 'BESS', 'BGTG', 'BOLA', 'ALII', 'ATIC', 'ACST', 'ALDO', 'FOOD', 'BALI', 'HOPE', 'ERAA', 'CBUT', 'DNAR', 'FIMP', 'DMND', 'BATR', 'BBSS', 'ATLA', 'GAMA', 'EDGE', 'BSIM', 'AKPI', 'HATM', 'GOLF', 'CGAS', 'HILL', 'HYGN', 'HRUM', 'COCO', 'ASLC', 'DADA', 'CCSI', 'AMIN', 'BRIS', 'ALTO', 'CBMF', 'HAJJ', 'DIVA', 'ELPI', 'HDFA', 'BHAT', 'HKMU', 'HAIS', 'BOGA', 'AVIA', 'AGAR', 'CITY', 'BABY', 'CHEM', 'CANI', 'AWAN', 'GOOD', 'AEGS', 'BAUT', 'ELIT', 'DEWI', 'ASPI', 'DATA', 'ACRO', 'BEEF', 'HOKI', 'DFAM', 'BMAS', 'FORZ', 'CMRY', 'FLMC', 'CLAY', 'ERAL', 'CAKK', 'CASA', 'GOLL', 'BLES', 'BUAH', 'GULA', 'BPII', 'GLOB', 'AREA', 'CLEO', 'BSBK',

In [28]:
from typing import List, Dict
from tqdm import tqdm
import requests

def get_tic_data_in_range(tic: str, start_date: str, end_date: str):
    """
    Fetches trading data for a specific ticker within a date range.

    Args:
        tic (str): The ticker symbol.
        start_date (str): The start date in YYYY-MM-DD format.
        end_date (str): The end date in YYYY-MM-DD format.

    Returns:
        dict: Trading data for the ticker in the given date range, or an empty dict if the request fails.
    """
    url = 'http://10.192.1.245:8080/orderbook/basic-trading-data'
    headers = {'Content-Type': 'application/json'}
    data = {
        "secCode": tic,
        "startDate": start_date,
        "endDate": end_date
    }

    items = []
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        items = response.json().get("data", [])
    except requests.RequestException as e:
        print(f"Error fetching trading data for {tic} on {start_date}: {e}")
        return []
    return items

def fix_missing_dates(tic: str, current_df: pd.DataFrame, working_dates: List[str], date2day_index: Dict):
    # TODO:
    current_dates = sorted(current_df["date"].unique())
    missing_dates = [d for d in working_dates if d not in current_dates]
    print(f"Processing missing dates in {tic}..")
    for missing_date in tqdm(missing_dates, total=len(missing_dates)):
        data = get_tic_data_in_range(tic, missing_date, missing_date)
        if not data:
            print(f"Failed to fetch {tic} data for date: {missing_date}")
        data = data[0]
        row = {
            "date": missing_date,
            "open": data.get("openPrice"),
            "high": data.get("highPrice"),
            "low": data.get("lowPrice"),
            "close": data.get("closePrice"),
            "volume": data.get("totalVolume"),
            "tic": tic,
            "day": date2day_index[missing_date]
        }
        # rows.append(row)
        RESET = False
        START_DATE = "2010-01-01"
        END_DATE = "2024-11-26"
        CSV_DIR = "/home/devmiftahul/trading_model/from_finrl-tutorials_git/tuntun_scripts/csv"
        FILENAME = f"{CSV_DIR}/{tic}_api_data_{START_DATE}_{END_DATE}.csv"
        new_data = pd.DataFrame([row])
        # if (existing_data.empty and not os.path.exists(FILENAME)) or RESET:
        if (not os.path.exists(FILENAME)) or RESET:
            RESET = False
            print(f"FILE NOT FOUND:\n{FILENAME}")
            # new_data.to_csv(FILENAME, index=False)
        else:
            new_data.to_csv(FILENAME, mode='a', header=False, index=False)

In [32]:
for tic in list_to_redo_fetch[:1]:
    current_path = fpath.replace("<TIC>", tic)
    current_df = pd.read_csv(current_path)
    print(f"Total row in {tic}: {len(current_df)}")
    current_dates = sorted(current_df["date"].unique().tolist())
    print(f"Earliest day to trade {tic}: {current_dates[0]}")
    print(f"Currently, {tic} only has {len(current_dates)} out of {len(working_dates)} days")
    print(f"Missing {len(working_dates)-len(current_dates)} days")
    fix_missing_dates(tic, current_df, working_dates, date2day_index)
    # print(current_df)

Total row in IBFN: 2408
Earliest day to trade IBFN: 2014-12-22
Currently, IBFN only has 2408 out of 3627 days
Missing 1219 days
Processing missing dates in IBFN..


  0%|                                                                                                                                                  | 0/1219 [00:00<?, ?it/s]

Failed to fetch IBFN data for date: 2010-01-04


IndexError: list index out of range